
-- RS de Atendimento: 615258

-- DATA 06/10/2020

-- OBJETIVO: Particionar a tabela dlkmumps.[tb_certificadosClb], pois a mesma tem a expectativa de 20 anos de dados e com isso a possibilidade de mais de 100 milhões de registros

-- QUANTIDADE DE LINHAS POR FRONTEIRA DE DADOS: 30 MILHÕES.

-- CAUSA RAIZ: Devido ao grande volume de dados, queremos dar a possibilidade do Attunity fazer a carga Full utilizando do paralelismo, que somente é possível por meio do particionamento


-- Código abaixo representa a quantidade de linhas por tabela no dia 06-10-2020.

-- Organizado pelas 10 maiores tabelas do banco de dados até em então.

-- Importante habilitar inicialmente a opção de estatísticas incrementais para que o particionamento tire proveito das estatísticas criadas pelo SQL Server por fronteira.

In [0]:
USE [master]
GO
ALTER DATABASE [DLKMUMPS] SET AUTO_CREATE_STATISTICS ON(INCREMENTAL = ON)
GO


In [4]:
USE
DLKMUMPS
GO


SELECT TOP 10
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
	and t.NAME = 'tb_coberturacertificadoClb'
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY 
    rows DESC, t.Name

	

(10 rows affected)

Total execution time: 00:00:00.1223191

TableName,SchemaName,rows,TotalSpaceKB,TotalSpaceMB,UsedSpaceKB,UsedSpaceMB,UnusedSpaceKB,UnusedSpaceMB
tb_coberturacertificadoClb,dlkmumps,12831289,1674312,"1635,07",1073264,"1048,11",601048,"586,96"
tb_certificadosClb,dlkmumps,5260290,3756720,"3668,67",3463440,"3382,27",293280,"286,41"
tb_historicomovimentacaoClb,dlkmumps,5247642,2416912,"2360,27",2218344,"2166,35",198568,"193,91"
tb_beneficiariosClb,dlkmumps,4719040,1425736,"1392,32",1236528,"1207,55",189208,"184,77"
tb_enderecosClb,dlkmumps,3026943,611208,"596,88",539808,"527,16",71400,"69,73"
tb_clientesClb,dlkmumps,2481548,1958568,"1912,66",1808928,"1766,53",149640,"146,13"
tb_certificadoassistenciasClb,dlkmumps,1820189,477768,"466,57",395032,"385,77",82736,"80,80"
tb_enderecosInd,dlkmumps,720182,158152,"154,45",140800,"137,50",17352,"16,95"
tb_clientesInd,dlkmumps,624270,493696,"482,13",465768,"454,85",27928,"27,27"
tb_coberturacertificadoVg,dlkmumps,418853,54728,"53,45",35048,"34,23",19680,"19,22"


A primeira tabela a ser particionada será a tb_coberturacertificadoClb, com um total de 12.831.289 milhões de registros.

Conforme definido em reunião, faremos a fronteira de partição de 30 milhões de registros cada.

Como obter o nome pk existente na tabela.

In [15]:

USE
DLKMUMPS
GO
    
    declare @databasename varchar(50)
	declare @tablename varchar(100)
		
	set @databasename = 'DLKMUMPS'
	set @tablename = 'tb_coberturacertificadoClb'

	SELECT cu.CONSTRAINT_NAME, cu.COLUMN_NAME 
	FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE cu 
	WHERE EXISTS ( SELECT tc.* FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc 
	WHERE tc.CONSTRAINT_CATALOG = @databasename 
	AND tc.TABLE_NAME = @tablename 
	AND tc.CONSTRAINT_TYPE = 'PRIMARY KEY' 
	AND tc.CONSTRAINT_NAME = cu.CONSTRAINT_NAME )








Commands completed successfully.

Total execution time: 00:00:00.0017260

(1 row affected)

Total execution time: 00:00:00.0836381

CONSTRAINT_NAME,COLUMN_NAME
PK_tb_coberturacertificadoClb,id_coberturacertificado


-- Retirar a tabela que faz parte do CDC
-- Como Verificar se a tabela está no CDC

In [52]:
SELECT s.name AS Schema_Name, tb.name AS Table_Name
, tb.object_id, tb.type, tb.type_desc, tb.is_tracked_by_cdc
FROM sys.tables tb
INNER JOIN sys.schemas s on s.schema_id = tb.schema_id
WHERE tb.is_tracked_by_cdc = 1
and s.name = 'dlkmumps'
and tb.name = 'tb_coberturacertificadoClb'

(1 row affected)

Total execution time: 00:00:00.9680407

Schema_Name,Table_Name,object_id,type,type_desc,is_tracked_by_cdc
dlkmumps,tb_coberturacertificadoClb,1397580017,U,USER_TABLE,1


-- Retirar a Tabela do processo de CDC

In [56]:
-- Verificar tabela com  CDC

USE
DLKMUMPS
GO

--EXEC sys.sp_cdc_help_change_data_capture
--GO
 
SELECT capture_instance as NomeTabelaCDC, OBJECT_NAME(source_object_id) as NomeTabelaDataSource, 


LEFT(
        capture_instance, 
        CHARINDEX('_', capture_instance) -1        
    ) schema_production
    
FROM cdc.change_tables
where OBJECT_NAME(source_object_id) = 'tb_coberturacertificadoClb'



Commands completed successfully.

Total execution time: 00:00:00.0323408

(1 row affected)

Total execution time: 00:00:00.1274693

NomeTabelaCDC,NomeTabelaDataSource,schema_production
dlkmumps_tb_coberturacertificadoClb,tb_coberturacertificadoClb,dlkmumps


In [70]:

use
DLKMUMPS
GO

--Remover tabela com CDC
EXEC sys.sp_cdc_disable_table
    @source_schema = 'dlkmumps', -- sysname
    @source_name = 'tb_coberturacertificadoClb', -- sysname
    @capture_instance = 'dlkmumps_tb_coberturacertificadoClb' -- sysname



Commands completed successfully.

Total execution time: 00:00:00.0226167

Commands completed successfully.

Total execution time: 00:00:00.7591148

--Verificar se de fato a tabela saiu do CDC


In [71]:
SELECT s.name AS Schema_Name, tb.name AS Table_Name
, tb.object_id, tb.type, tb.type_desc, tb.is_tracked_by_cdc
FROM sys.tables tb
INNER JOIN sys.schemas s on s.schema_id = tb.schema_id
WHERE tb.is_tracked_by_cdc = 1
and s.name = 'dlkmumps'
and tb.name = 'tb_coberturacertificadoClb'

(0 rows affected)

Total execution time: 00:00:00.0416852

Schema_Name,Table_Name,object_id,type,type_desc,is_tracked_by_cdc


-- Comando para fazer o drop da pk da tabela a ser particionada

In [59]:
USE [DLKMUMPS]
GO

/****** Object:  Index [PK_tb_coberturacertificadoClb]    Script Date: 07/10/2020 11:40:27 ******/
ALTER TABLE [dlkmumps].[tb_coberturacertificadoClb] DROP CONSTRAINT [PK_tb_coberturacertificadoClb] WITH ( ONLINE = OFF )
GO


Commands completed successfully.

Total execution time: 00:00:00.0047385

Commands completed successfully.

Total execution time: 00:00:00.2066177

Validar a existência de indices não cluster na tabela que vai ser particionada.

In [14]:
/*****************************************************************************
MIT License, http://www.opensource.org/licenses/mit-license.php
Contact: help@sqlworkbooks.com
Copyright (c) 2018 SQL Workbooks LLC
Permission is hereby granted, free of charge, to any person 
obtaining a copy of this software and associated documentation
files (the "Software"), to deal in the Software without 
restriction, including without limitation the rights to use,
copy, modify, merge, publish, distribute, sublicense, and/or
sell copies of the Software, and to permit persons to whom 
the Software is furnished to do so, subject to the following 
conditions:
The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING 
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR 
OTHER DEALINGS IN THE SOFTWARE.
*****************************************************************************/
USE
DLKMUMPS
GO


declare @tablename varchar(50)
declare @schemaname varchar(50)

set @tablename = 'tb_coberturacertificadoClb'
set @schemaname = 'dlkmumps'

SELECT 
    DB_NAME() AS database_name,
    sc.name + N'.' + t.name AS table_name,
    (SELECT MAX(user_reads) 
        FROM (VALUES (last_user_seek), (last_user_scan), (last_user_lookup)) AS value(user_reads)) AS last_user_read,
    last_user_update,
    CASE si.index_id WHEN 0 THEN N'/* No create statement (Heap) */'
    ELSE 
        CASE is_primary_key WHEN 1 THEN
            N'ALTER TABLE ' + QUOTENAME(sc.name) + N'.' + QUOTENAME(t.name) + N' ADD CONSTRAINT ' + QUOTENAME(si.name) + N' PRIMARY KEY ' +
                CASE WHEN si.index_id > 1 THEN N'NON' ELSE N'' END + N'CLUSTERED '
            ELSE N'CREATE ' + 
                CASE WHEN si.is_unique = 1 then N'UNIQUE ' ELSE N'' END +
                CASE WHEN si.index_id > 1 THEN N'NON' ELSE N'' END + N'CLUSTERED ' +
                N'INDEX ' + QUOTENAME(si.name) + N' ON ' + QUOTENAME(sc.name) + N'.' + QUOTENAME(t.name) + N' '
        END +
        /* key def */ N'(' + key_definition + N')' +
        /* includes */ CASE WHEN include_definition IS NOT NULL THEN 
            N' INCLUDE (' + include_definition + N')'
            ELSE N''
        END +
        /* filters */ CASE WHEN filter_definition IS NOT NULL THEN 
            N' WHERE ' + filter_definition ELSE N''
        END +
        /* with clause - compression goes here */
        CASE WHEN row_compression_partition_list IS NOT NULL OR page_compression_partition_list IS NOT NULL 
            THEN N' WITH (' +
                CASE WHEN row_compression_partition_list IS NOT NULL THEN
                    N'DATA_COMPRESSION = ROW ' + CASE WHEN psc.name IS NULL THEN N'' ELSE + N' ON PARTITIONS (' + row_compression_partition_list + N')' END
                ELSE N'' END +
                CASE WHEN row_compression_partition_list IS NOT NULL AND page_compression_partition_list IS NOT NULL THEN N', ' ELSE N'' END +
                CASE WHEN page_compression_partition_list IS NOT NULL THEN
                    N'DATA_COMPRESSION = PAGE ' + CASE WHEN psc.name IS NULL THEN N'' ELSE + N' ON PARTITIONS (' + page_compression_partition_list + N')' END
                ELSE N'' END
            + N')'
            ELSE N''
        END +
        /* ON where? filegroup? partition scheme? */
        ' ON ' + CASE WHEN psc.name is null 
            THEN ISNULL(QUOTENAME(fg.name),N'')
            ELSE psc.name + N' (' + partitioning_column.column_name + N')' 
            END
        + N';'
    END AS index_create_statement,
    si.index_id,
    si.name AS index_name,
    partition_sums.reserved_in_row_GB,
    partition_sums.reserved_LOB_GB,
    partition_sums.row_count,
    stat.user_seeks,
    stat.user_scans,
    stat.user_lookups,
    user_updates AS queries_that_modified,
    partition_sums.partition_count,
    si.allow_page_locks,
    si.allow_row_locks,
    si.is_hypothetical,
    si.has_filter,
    si.fill_factor,
    si.is_unique,
    ISNULL(pf.name, '/* Not partitioned */') AS partition_function,
    ISNULL(psc.name, fg.name) AS partition_scheme_or_filegroup,
    t.create_date AS table_created_date,
    t.modify_date AS table_modify_date
FROM sys.indexes AS si
JOIN sys.tables AS t ON si.object_id=t.object_id
JOIN sys.schemas AS sc ON t.schema_id=sc.schema_id
LEFT JOIN sys.dm_db_index_usage_stats AS stat ON 
    stat.database_id = DB_ID() 
    and si.object_id=stat.object_id 
    and si.index_id=stat.index_id
LEFT JOIN sys.partition_schemes AS psc ON si.data_space_id=psc.data_space_id
LEFT JOIN sys.partition_functions AS pf ON psc.function_id=pf.function_id
LEFT JOIN sys.filegroups AS fg ON si.data_space_id=fg.data_space_id
/* Key list */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + QUOTENAME(c.name) +
        CASE ic.is_descending_key WHEN 1 then N' DESC' ELSE N'' END
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.key_ordinal > 0
    ORDER BY ic.key_ordinal FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS keys ( key_definition )
/* Partitioning Ordinal */ OUTER APPLY (
    SELECT MAX(QUOTENAME(c.name)) AS column_name
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.partition_ordinal = 1) AS partitioning_column
/* Include list */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + QUOTENAME(c.name)
    FROM sys.index_columns AS ic 
    JOIN sys.columns AS c ON 
        ic.column_id=c.column_id  
        and ic.object_id=c.object_id
    WHERE ic.object_id = si.object_id
        and ic.index_id=si.index_id
        and ic.is_included_column = 1
    ORDER BY c.name FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS includes ( include_definition )
/* Partitions */ OUTER APPLY ( 
    SELECT 
        COUNT(*) AS partition_count,
        CAST(SUM(ps.in_row_reserved_page_count)*8./1024./1024. AS NUMERIC(32,1)) AS reserved_in_row_GB,
        CAST(SUM(ps.lob_reserved_page_count)*8./1024./1024. AS NUMERIC(32,1)) AS reserved_LOB_GB,
        SUM(ps.row_count) AS row_count
    FROM sys.partitions AS p
    JOIN sys.dm_db_partition_stats AS ps ON
        p.partition_id=ps.partition_id
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
    ) AS partition_sums
/* row compression list by partition */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + CAST(p.partition_number AS VARCHAR(32))
    FROM sys.partitions AS p
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
        and p.data_compression = 1
    ORDER BY p.partition_number FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS row_compression_clause ( row_compression_partition_list )
/* data compression list by partition */ OUTER APPLY ( SELECT STUFF (
    (SELECT N', ' + CAST(p.partition_number AS VARCHAR(32))
    FROM sys.partitions AS p
    WHERE p.object_id = si.object_id
        and p.index_id=si.index_id
        and p.data_compression = 2
    ORDER BY p.partition_number FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'),1,2,'')) AS page_compression_clause ( page_compression_partition_list )
WHERE 
    si.type IN (0,1,2) /* heap, clustered, nonclustered */
	and t.name = @tablename
	and sc.name = @schemaname
ORDER BY table_name, si.index_id
    OPTION (RECOMPILE);
GO

Commands completed successfully.

Total execution time: 00:00:00.0053528

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.3884597

database_name,table_name,last_user_read,last_user_update,index_create_statement,index_id,index_name,reserved_in_row_GB,reserved_LOB_GB,row_count,user_seeks,user_scans,user_lookups,queries_that_modified,partition_count,allow_page_locks,allow_row_locks,is_hypothetical,has_filter,fill_factor,is_unique,partition_function,partition_scheme_or_filegroup,table_created_date,table_modify_date
DLKMUMPS,dlkmumps.tb_coberturacertificadoClb,2020-10-07 10:47:02.017,NULL,ALTER TABLE [dlkmumps].[tb_coberturacertificadoClb] ADD CONSTRAINT [PK_tb_coberturacertificadoClb] PRIMARY KEY CLUSTERED ([id_coberturacertificado]) ON [PRIMARY];,1,PK_tb_coberturacertificadoClb,"1,6","0,0",12831289,0,7,0,0,1,1,1,0,0,0,1,/* Not partitioned */,PRIMARY,2020-09-22 19:25:38.240,2020-10-06 16:43:26.753


-- ADICIONAR OS FILEGROUPS PARA O PARTICIONAMENTO EM ARQUIVOS FISICOS DIFERENTES    

In [11]:
use [master]

GO
USE [master]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_01]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_02]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_03]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_04]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_05]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_06]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_07]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_08]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_09]
GO
ALTER DATABASE [DLKMUMPS] ADD FILEGROUP [FG_Particionamento_10]
GO
    

Commands completed successfully.

Total execution time: 00:00:00.0012364

Commands completed successfully.

Total execution time: 00:00:00.0008227

Commands completed successfully.

...

Total execution time: 00:00:00.0232822

Commands completed successfully.

Total execution time: 00:00:00.0118183

Commands completed successfully.

Total execution time: 00:00:00.0043615

-- Adicionar os arquivos fisicos que farão parte do particionamento

In [30]:

use [DLKMUMPS]

GO
use [model]

GO
USE [master]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_01', FILENAME = N'E:\Dados\File_Particionamento_01.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_01]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_02', FILENAME = N'E:\Dados\File_Particionamento_02.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_02]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_03', FILENAME = N'E:\Dados\File_Particionamento_03.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_03]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_04', FILENAME = N'E:\Dados\File_Particionamento_04.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_04]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_05', FILENAME = N'E:\Dados\File_Particionamento_05.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_05]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_06', FILENAME = N'E:\Dados\File_Particionamento_06.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_06]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_07', FILENAME = N'E:\Dados\File_Particionamento_07.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_07]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_08', FILENAME = N'E:\Dados\File_Particionamento_08.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_08]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_09', FILENAME = N'E:\Dados\File_Particionamento_09.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_09]
GO
ALTER DATABASE [DLKMUMPS] ADD FILE ( NAME = N'File_Particionamento_10', FILENAME = N'E:\Dados\File_Particionamento_10.ndf' , SIZE = 8192KB , FILEGROWTH = 1048576KB ) TO FILEGROUP [FG_Particionamento_10]
GO



Commands completed successfully.

Total execution time: 00:00:00.0169403

Commands completed successfully.

Total execution time: 00:00:00.0032515

Commands completed successfully.

...

Total execution time: 00:00:00.1605541

: Msg 1828, Level 16, State 4, Line 25
The logical file name "File_Particionamento_09" is already in use. Choose a different name.

Total execution time: 00:00:00.0020410

: Msg 1828, Level 16, State 4, Line 27
The logical file name "File_Particionamento_10" is already in use. Choose a different name.

Total execution time: 00:00:00.0018404

-- Verificar a tipagem do campo da coluna que fará parte da Função de Partição.

In [31]:
USE
DLKMUMPS
GO

SELECT @@SERVERNAME as ServidorAzure, DB_NAME() as DatabaseName,
TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,TABLE_SCHEMA
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME  = 'tb_coberturacertificadoClb'
and TABLE_SCHEMA = 'dlkmumps'
GO

sp_help 'dlkmumps.tb_coberturacertificadoClb'



Commands completed successfully.

Total execution time: 00:00:00.0014205

(5 rows affected)

Total execution time: 00:00:00.0095258

No foreign keys reference table 'dlkmumps.tb_coberturacertificadoClb', or you do not have permissions on referencing tables.

...

Identity,Seed,Increment,Not For Replication
id_coberturacertificado,1,1,1


RowGuidCol
No rowguidcol column defined.


Data_located_on_filegroup
PRIMARY


index_name,index_description,index_keys
PK_tb_coberturacertificadoClb,"clustered, unique, primary key located on PRIMARY",id_coberturacertificado


constraint_type,constraint_name,delete_action,update_action,status_enabled,status_for_replication,constraint_keys
PRIMARY KEY (clustered),PK_tb_coberturacertificadoClb,(n/a),(n/a),(n/a),(n/a),id_coberturacertificado


-- Criação da Função de Partição Aqui seria importante criar uma nomenclatura que faça lembrar o nome da tabela.

In [60]:
USE
DLKMUMPS
GO



CREATE PARTITION FUNCTION [PF_Int_Partition_tb_coberturacertificadoClb] (int)
AS RANGE LEFT FOR VALUES 
(
50000000,
100000000,
150000000,
200000000,
250000000,
300000000,
350000000,
400000000,
450000000
)


Commands completed successfully.

Total execution time: 00:00:00.0033800

Commands completed successfully.

Total execution time: 00:00:00.0055515

-- Criar a função de Schema

In [61]:
USE
DLKMUMPS
GO

CREATE PARTITION SCHEME [PS_Int_Partition_tb_coberturacertificadoClb]
AS PARTITION [PF_Int_Partition_tb_coberturacertificadoClb]
TO
(
    [FG_Particionamento_01],
    [FG_Particionamento_02],
    [FG_Particionamento_03],
    [FG_Particionamento_04],
    [FG_Particionamento_05],
    [FG_Particionamento_06],
    [FG_Particionamento_07],
    [FG_Particionamento_08],
    [FG_Particionamento_09],
    [FG_Particionamento_10]
)

Commands completed successfully.

Total execution time: 00:00:00.0018352

Commands completed successfully.

Total execution time: 00:00:00.0090583

-- Criação da PK com Indice Cluster já na função de partição

-- Atenção para a mudança do Partition Schema que deve ser criado, bem como o nome do campo.

In [62]:

USE
DLKMUMPS
GO

/****** Object:  Index [PK_tb_coberturacertificadoClb]    Script Date: 07/10/2020 10:42:12 ******/
ALTER TABLE [dlkmumps].[tb_coberturacertificadoClb] ADD  CONSTRAINT [PK_tb_coberturacertificadoClb] PRIMARY KEY CLUSTERED 
(
	[id_coberturacertificado] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PS_Int_Partition_tb_coberturacertificadoClb]
(id_coberturacertificado)
GO



Commands completed successfully.

Total execution time: 00:00:00.0013778

Commands completed successfully.

Total execution time: 00:00:56.1406859

-- Habilitar as estatisticas incrementais na tabela com o nome indice

In [64]:
USE
DLKMUMPS
GO
ALTER INDEX PK_tb_coberturacertificadoClb ON [dlkmumps].[tb_coberturacertificadoClb] REBUILD WITH (STATISTICS_INCREMENTAL=ON)

Commands completed successfully.

Total execution time: 00:00:00.0098485

Commands completed successfully.

Total execution time: 00:00:18.5720578

-- Verificar com que está as fronteiras dos indices

In [67]:
use
DLKMUMPS
GO


DECLARE @tablename varchar(50)
set @tablename = 'tb_coberturacertificadoClb'

DECLARE @schemaname varchar(50)

set @schemaname = 'dlkmumps'

SELECT s.NAME AS 'schema'
       , o.NAME AS 'table'
       , CASE o.type
             WHEN 'v' THEN 'View'
             WHEN 'u' THEN 'Table'
             ELSE o.type
         END AS objecttype
       , i.NAME AS indexname
       , i.type_desc
       , p.data_compression_desc
       , ds.type_desc AS DataSpaceTypeDesc
       , p.partition_number
       , pf.NAME AS pf_name
       , ps.NAME AS ps_name
       , CASE
             WHEN partitionds.NAME IS NULL THEN ds.NAME
             ELSE partitionds.NAME
         END AS partition_fg
       , i.is_primary_key
       , i.is_unique
       , p.rows
FROM   sys.indexes i
       INNER JOIN sys.objects o
               ON o.object_id = i.object_id
       INNER JOIN sys.data_spaces ds
               ON DS.data_space_id = i.data_space_id
       LEFT JOIN sys.schemas s
              ON o.schema_id = s.schema_id
       LEFT JOIN sys.partitions p
              ON i.index_id  = p.index_id
             AND i.object_id = p.object_id
       LEFT JOIN sys.destination_data_spaces dds
              ON i.data_space_id    = dds.partition_scheme_id
             AND p.partition_number = dds.destination_id
       LEFT JOIN sys.data_spaces partitionds
              ON dds.data_space_id = partitionds.data_space_id
       LEFT JOIN sys.partition_schemes AS ps
              ON dds.partition_scheme_id = ps.data_space_id
       LEFT JOIN sys.partition_functions AS pf
              ON ps.function_id = pf.function_id
WHERE  o.NAME = @tablename
and    s.NAME = @schemaname
ORDER  BY s.NAME
          , o.NAME
          , i.NAME
          , p.partition_number
GO

Commands completed successfully.

Total execution time: 00:00:00.0028434

(10 rows affected)

Total execution time: 00:00:00.0728216

schema,table,objecttype,indexname,type_desc,data_compression_desc,DataSpaceTypeDesc,partition_number,pf_name,ps_name,partition_fg,is_primary_key,is_unique,rows
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,1,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_01,1,1,12831289
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,2,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_02,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,3,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_03,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,4,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_04,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,5,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_05,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,6,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_06,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,7,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_07,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,8,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_08,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,9,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_09,1,1,0
dlkmumps,tb_coberturacertificadoClb,Table,PK_tb_coberturacertificadoClb,CLUSTERED,NONE,PARTITION_SCHEME,10,PF_Int_Partition_tb_coberturacertificadoClb,PS_Int_Partition_tb_coberturacertificadoClb,FG_Particionamento_10,1,1,0


-- Voltar com a tabela original para o processo de CDC

-- Referência importante, pois em situações onde for usada o partition Switch, Merge, etc, pode acarretar em inconsistências de dados sendo necessário refazer o CDC.

https://docs.microsoft.com/pt-br/sql/relational-databases/system-stored-procedures/sys-sp-cdc-enable-table-transact-sql?view=sql-server-ver15   

In [72]:
EXEC sys.sp_cdc_enable_table  
@source_schema = N'dlkmumps',  
@source_name   = N'tb_coberturacertificadoClb',  
@role_name     = NULL,
@supports_net_changes = 1  


Warning: The @allow_partition_switch parameter is set to 1. Change data capture will not track changes introduced in the table resulting from a partition switch which will cause data inconsistency when changes are consumed. Refer to books online for more information about partition switching behavior when using Change Data Capture.

Total execution time: 00:00:00.2081170

-- Query para avaliar o partition elimination

Verificar por meio do plano de execução o partition elimination 

In [0]:
select id_coberturacertificado from dlkmumps.tb_coberturacertificadoClb
where id_coberturacertificado = 80000000